In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot

In [6]:
def read_sctp_log(file):
    with open(file) as f:
        for line in f:
            if 'cwnd=' in line:
                print(line)

In [7]:
read_sctp_log('../pion/offer.log')

sctp TRACE: 17:16:02.305243 association.go:374: [0xc0004001e0] updated cwnd=4380 ssthresh=0 inflight=0 (INI)

sctp TRACE: 17:16:02.325896 association.go:1280: [0xc0004001e0] updated cwnd=4380 ssthresh=1048576 inflight=0 (INI)

sctp TRACE: 17:16:02.400573 association.go:1694: [0xc0004001e0] updated cwnd=5596 ssthresh=1048576 acked=1216 (SS)

sctp TRACE: 17:16:02.400798 association.go:1694: [0xc0004001e0] updated cwnd=7996 ssthresh=1048576 acked=2400 (SS)

sctp TRACE: 17:16:02.420992 association.go:1697: [0xc0004001e0] cwnd did not grow: cwnd=7996 ssthresh=1048576 acked=2400 FR=false pending=0

sctp TRACE: 17:16:02.421410 association.go:1697: [0xc0004001e0] cwnd did not grow: cwnd=7996 ssthresh=1048576 acked=4240 FR=false pending=0

sctp TRACE: 17:16:02.454378 association.go:1694: [0xc0004001e0] updated cwnd=11596 ssthresh=1048576 acked=3600 (SS)

sctp TRACE: 17:16:02.454588 association.go:1697: [0xc0004001e0] cwnd did not grow: cwnd=11596 ssthresh=1048576 acked=3600 FR=false pending=0

